<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/BDSN_ML_Housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![CC-BY-SA](https://licensebuttons.net/l/by-sa/3.0/88x31.png)<br>
<hr>

![alt text](http://1.bp.blogspot.com/-nqAGzznZQNo/UwS8rxjfXeI/AAAAAAAABTA/nunmRLowpps/s1600/PraxisLogo.gif)<br>
[Data Science Program](http://praxis.ac.in/Programs/business-analytics/)

<hr>

[Prithwis Mukerjee](http://www.yantrajaal.com)<br>

# BDS Housing - Machine Learning <br>
https://www.datacamp.com/community/tutorials/apache-spark-tutorial-machine-learning

# Initialise

## Deprecated -- do not use

In [ ]:
!apt-get update > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://apache.osuosl.org/spark/spark-2.2.2/spark-2.2.2-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
#wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!wget -q http://apache.osuosl.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
#
# if the current version of Spark is not used, there may be errors
# check here for current versions http://apache.osuosl.org/spark
#
#!tar xf spark-2.4.0-bin-hadoop2.7.tgz
#!tar xf spark-2.4.5-bin-hadoop2.7.tgz
#!tar xf spark-2.4.6-bin-hadoop2.7.tgz
#!tar xf spark-3.0.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
#!pip install -q findspark
!pip install -q pyspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

## Simple Install 2022

In [1]:
#import findspark
#findspark.init()
!pip3 install -q pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
#sc

     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 63.7 MB/s 


# Load Data <br>
Data is available at https://drive.google.com/drive/folders/1Xj1YY9DE19-N-EvOtI_sfkozds-6fLG_?usp=sharing

In [2]:
# Original Data
#!wget http://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.tgz
# Data from Google Drive
!gdown https://drive.google.com/uc?id=1kizAZ1XPf-eyiSCo8SiDKgyOxgYkm9A5
!gdown https://drive.google.com/uc?id=1tQvUiBdTu1p-JJwHj472ruTtpfOWwvKn


Downloading...
From: https://drive.google.com/uc?id=1kizAZ1XPf-eyiSCo8SiDKgyOxgYkm9A5
To: /content/housing-header.txt
100% 127/127 [00:00<00:00, 200kB/s]
Downloading...
From: https://drive.google.com/uc?id=1tQvUiBdTu1p-JJwHj472ruTtpfOWwvKn
To: /content/housing-data.txt
100% 4.69M/4.69M [00:00<00:00, 23.4MB/s]


In [ ]:
# if pulling original data
#!tar xf cal_housing.tgz


In [ ]:
#!ls -al

In [3]:
# Load in the header
header = sc.textFile('housing-header.txt').flatMap(lambda l:l.split(","))

In [4]:
header.collect()

['median house value',
 ' median income',
 ' housing median age',
 ' total rooms',
 ' total bedrooms',
 ' population',
 ' households',
 ' latitude',
 ' longitude']

In [5]:
housingRDD = sc.textFile('housing-data.txt').map(lambda l:l.split())
#housingRDD = sc.textFile('CaliforniaHousing/cal_housing.data').map(lambda l:l.split())

In [6]:
housingRDD.take(2)

[['4.5260000000000000e+005',
  '8.3252000000000006e+000',
  '4.1000000000000000e+001',
  '8.8000000000000000e+002',
  '1.2900000000000000e+002',
  '3.2200000000000000e+002',
  '1.2600000000000000e+002',
  '3.7880000000000003e+001',
  '-1.2223000000000000e+002'],
 ['3.5850000000000000e+005',
  '8.3013999999999992e+000',
  '2.1000000000000000e+001',
  '7.0990000000000000e+003',
  '1.1060000000000000e+003',
  '2.4010000000000000e+003',
  '1.1380000000000000e+003',
  '3.7859999999999999e+001',
  '-1.2222000000000000e+002']]

In [7]:
# Import the necessary modules 
from pyspark.sql import Row

In [9]:
# Map the RDD to a DF
housing_df = housingRDD.map(lambda line: Row(medianHouseValue=float(line[0]), 
                              medianIncome=float(line[1]), 
                              housingMedianAge=float(line[2]),
                              totalRooms=float(line[3]),
                              totalBedrooms=float(line[4]),
                              population=float(line[5]), 
                              households=float(line[6]),
                              latitude=float(line[7]),
                              longitude=float(line[8]))).toDF()

In [10]:
housing_df.show()

+----------------+------------+----------------+----------+-------------+----------+----------+--------+---------+
|medianHouseValue|medianIncome|housingMedianAge|totalRooms|totalBedrooms|population|households|latitude|longitude|
+----------------+------------+----------------+----------+-------------+----------+----------+--------+---------+
|        452600.0|      8.3252|            41.0|     880.0|        129.0|     322.0|     126.0|   37.88|  -122.23|
|        358500.0|      8.3014|            21.0|    7099.0|       1106.0|    2401.0|    1138.0|   37.86|  -122.22|
|        352100.0|      7.2574|            52.0|    1467.0|        190.0|     496.0|     177.0|   37.85|  -122.24|
|        341300.0|      5.6431|            52.0|    1274.0|        235.0|     558.0|     219.0|   37.85|  -122.25|
|        342200.0|      3.8462|            52.0|    1627.0|        280.0|     565.0|     259.0|   37.85|  -122.25|
|        269700.0|      4.0368|            52.0|     919.0|        213.0|     41

In [11]:
housing_df.dtypes

[('medianHouseValue', 'double'),
 ('medianIncome', 'double'),
 ('housingMedianAge', 'double'),
 ('totalRooms', 'double'),
 ('totalBedrooms', 'double'),
 ('population', 'double'),
 ('households', 'double'),
 ('latitude', 'double'),
 ('longitude', 'double')]

In [12]:
housing_df.printSchema()

root
 |-- medianHouseValue: double (nullable = true)
 |-- medianIncome: double (nullable = true)
 |-- housingMedianAge: double (nullable = true)
 |-- totalRooms: double (nullable = true)
 |-- totalBedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



# Data Exploration

In [13]:
housing_df.select('population','totalBedRooms').show(10)

+----------+-------------+
|population|totalBedRooms|
+----------+-------------+
|     322.0|        129.0|
|    2401.0|       1106.0|
|     496.0|        190.0|
|     558.0|        235.0|
|     565.0|        280.0|
|     413.0|        213.0|
|    1094.0|        489.0|
|    1157.0|        687.0|
|    1206.0|        665.0|
|    1551.0|        707.0|
+----------+-------------+
only showing top 10 rows



In [14]:
housing_df.groupBy("housingMedianAge").count().sort("housingMedianAge",ascending=False).show()

+----------------+-----+
|housingMedianAge|count|
+----------------+-----+
|            52.0| 1273|
|            51.0|   48|
|            50.0|  136|
|            49.0|  134|
|            48.0|  177|
|            47.0|  198|
|            46.0|  245|
|            45.0|  294|
|            44.0|  356|
|            43.0|  353|
|            42.0|  368|
|            41.0|  296|
|            40.0|  304|
|            39.0|  369|
|            38.0|  394|
|            37.0|  537|
|            36.0|  862|
|            35.0|  824|
|            34.0|  689|
|            33.0|  615|
+----------------+-----+
only showing top 20 rows



In [15]:
housing_df.describe().show()

+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+
|summary|  medianHouseValue|      medianIncome|  housingMedianAge|        totalRooms|     totalBedrooms|        population|        households|          latitude|          longitude|
+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+
|  count|             20640|             20640|             20640|             20640|             20640|             20640|             20640|             20640|              20640|
|   mean|206855.81690891474| 3.870671002906978|28.639486434108527|2635.7630813953488| 537.8980135658915|1425.4767441860465| 499.5396802325581| 35.63186143410859|-119.56970445736413|
| stddev|115395.61587441381|1.8998217179452654|12.585557612111637|2181.6152515827985|421.2

# Data Pre-processing

In [16]:
housing_df.take(2)

[Row(medianHouseValue=452600.0, medianIncome=8.3252, housingMedianAge=41.0, totalRooms=880.0, totalBedrooms=129.0, population=322.0, households=126.0, latitude=37.88, longitude=-122.23),
 Row(medianHouseValue=358500.0, medianIncome=8.3014, housingMedianAge=21.0, totalRooms=7099.0, totalBedrooms=1106.0, population=2401.0, households=1138.0, latitude=37.86, longitude=-122.22)]

In [17]:
# Import all from `sql.functions` 
from pyspark.sql.functions import *

# Adjust the values of `medianHouseValue`
housing_df = housing_df.withColumn("medianHouseValue", col("medianHouseValue")/100000)

# Show the first 2 lines of `df`
housing_df.take(2)

[Row(medianHouseValue=4.526, medianIncome=8.3252, housingMedianAge=41.0, totalRooms=880.0, totalBedrooms=129.0, population=322.0, households=126.0, latitude=37.88, longitude=-122.23),
 Row(medianHouseValue=3.585, medianIncome=8.3014, housingMedianAge=21.0, totalRooms=7099.0, totalBedrooms=1106.0, population=2401.0, households=1138.0, latitude=37.86, longitude=-122.22)]

# Feature Engineering

In [18]:
# Import all from `sql.functions` if you haven't yet
from pyspark.sql.functions import *

# Divide `totalRooms` by `households`
roomsPerHousehold = housing_df.select(col("totalRooms")/col("households"))

# Divide `population` by `households`
populationPerHousehold = housing_df.select(col("population")/col("households"))

# Divide `totalBedRooms` by `totalRooms`
bedroomsPerRoom = housing_df.select(col("totalBedRooms")/col("totalRooms"))

# Add the new columns to `df`
housing_df = housing_df.withColumn("roomsPerHousehold", col("totalRooms")/col("households")) \
   .withColumn("populationPerHousehold", col("population")/col("households")) \
   .withColumn("bedroomsPerRoom", col("totalBedRooms")/col("totalRooms"))
   
# Inspect the result
housing_df.first()


Row(medianHouseValue=4.526, medianIncome=8.3252, housingMedianAge=41.0, totalRooms=880.0, totalBedrooms=129.0, population=322.0, households=126.0, latitude=37.88, longitude=-122.23, roomsPerHousehold=6.984126984126984, populationPerHousehold=2.5555555555555554, bedroomsPerRoom=0.14659090909090908)

In [19]:
# Re-order and select columns
housing_df = housing_df.select("medianHouseValue", 
              "totalBedRooms", 
              "population", 
              "households", 
              "medianIncome", 
              "roomsPerHousehold", 
              "populationPerHousehold", 
              "bedroomsPerRoom")

In [20]:
# Import `DenseVector`
from pyspark.ml.linalg import DenseVector

# Define the `input_data` 
input_data = housing_df.rdd.map(lambda x: (x[0], DenseVector(x[1:])))

# Replace `df` with the new DataFrame
housing_df2 = spark.createDataFrame(input_data, ["label", "features"])

In [21]:
housing_df2.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|4.526|[129.0,322.0,126....|
|3.585|[1106.0,2401.0,11...|
|3.521|[190.0,496.0,177....|
|3.413|[235.0,558.0,219....|
|3.422|[280.0,565.0,259....|
|2.697|[213.0,413.0,193....|
|2.992|[489.0,1094.0,514...|
|2.414|[687.0,1157.0,647...|
|2.267|[665.0,1206.0,595...|
|2.611|[707.0,1551.0,714...|
|2.815|[434.0,910.0,402....|
|2.418|[752.0,1504.0,734...|
|2.135|[474.0,1098.0,468...|
|1.913|[191.0,345.0,174....|
|1.592|[626.0,1212.0,620...|
|  1.4|[283.0,697.0,264....|
|1.525|[347.0,793.0,331....|
|1.555|[293.0,648.0,303....|
|1.587|[455.0,990.0,419....|
|1.629|[298.0,690.0,275....|
+-----+--------------------+
only showing top 20 rows



In [22]:
# Import `StandardScaler` 
from pyspark.ml.feature import StandardScaler

# Initialize the `standardScaler`
standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled")

# Fit the DataFrame to the scaler
scaler = standardScaler.fit(housing_df2)

# Transform the data in `df` with the scaler
scaled_housing_df2 = scaler.transform(housing_df2)

In [23]:
# Inspect the result
scaled_housing_df2.take(2)

[Row(label=4.526, features=DenseVector([129.0, 322.0, 126.0, 8.3252, 6.9841, 2.5556, 0.1466]), features_scaled=DenseVector([0.3062, 0.2843, 0.3296, 4.3821, 2.8228, 0.2461, 2.5264])),
 Row(label=3.585, features=DenseVector([1106.0, 2401.0, 1138.0, 8.3014, 6.2381, 2.1098, 0.1558]), features_scaled=DenseVector([2.6255, 2.1202, 2.9765, 4.3696, 2.5213, 0.2031, 2.6851]))]

# Building ML Models

In [24]:
# Split the data into train and test sets
train_data, test_data = scaled_housing_df2.randomSplit([.8,.2],seed=1234)

In [25]:
train_data.show()

+-------+--------------------+--------------------+
|  label|            features|     features_scaled|
+-------+--------------------+--------------------+
|0.14999|[73.0,85.0,38.0,1...|[0.17329463000311...|
|0.14999|[267.0,628.0,225....|[0.63383104398397...|
|  0.175|[168.0,259.0,138....|[0.39881503891126...|
|   0.25|[33.0,64.0,27.0,0...|[0.07833866835757...|
|  0.266|[309.0,808.0,294....|[0.73353480371179...|
|  0.269|[543.0,1423.0,482...|[1.28902717933820...|
|    0.3|[183.0,500.0,177....|[0.43442352452834...|
|    0.3|[448.0,338.0,182....|[1.06350677043004...|
|  0.325|[49.0,51.0,20.0,4...|[0.11632105301578...|
|  0.329|[386.0,436.0,213....|[0.91632502987946...|
|  0.332|[131.0,511.0,124....|[0.31098077438914...|
|  0.344|[121.0,530.0,115....|[0.28724178397775...|
|  0.346|[208.0,660.0,188....|[0.49377100055680...|
|  0.366|[199.0,567.0,204....|[0.47240590918656...|
|  0.367|[238.0,425.0,157....|[0.56498797179096...|
|  0.375|[20.0,39.0,18.0,3...|[0.04747798082276...|
|  0.375|[19

In [26]:
# Import `LinearRegression`
from pyspark.ml.regression import LinearRegression

# Initialize `lr`
lr = LinearRegression(labelCol="label", maxIter=10, regParam=0.3, elasticNetParam=0.8)
#lr = LinearRegression(labelCol="label", featuresCol="features", maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the data to the model
linearModel = lr.fit(train_data)

In [27]:
# Generate predictions
predicted = linearModel.transform(test_data)
predicted.show()

+-------+--------------------+--------------------+------------------+
|  label|            features|     features_scaled|        prediction|
+-------+--------------------+--------------------+------------------+
|0.14999|[239.0,490.0,164....|[0.56736187083210...|1.5779571530300995|
|  0.225|[73.0,216.0,63.0,...|[0.17329463000311...|1.7357597669255935|
|  0.388|[263.0,1274.0,241...|[0.62433544781942...|1.5345134073263487|
|  0.394|[310.0,779.0,275....|[0.73590870275293...|1.3663369868130295|
|  0.396|[296.0,1228.0,289...|[0.70267411617699...|  1.29015262921757|
|  0.398|[316.0,672.0,241....|[0.75015209699976...|1.5260057881424178|
|    0.4|[185.0,318.0,115....|[0.43917132261062...|1.4635982848383564|
|  0.431|[523.0,1772.0,479...|[1.24154919851543...|1.3740212880114187|
|   0.44|[386.0,775.0,320....|[0.91632502987946...|1.3323065100773055|
|  0.441|[322.0,927.0,277....|[0.76439549124659...| 1.359695555062993|
|   0.45|[97.0,530.0,96.0,...|[0.23026820699043...| 1.276073891664807|
|  0.4

In [28]:
# Extract the predictions and the "known" correct labels
predictions = predicted.select("prediction").rdd.map(lambda x: x[0])
labels = predicted.select("label").rdd.map(lambda x: x[0])


In [29]:
predictions.take(5), labels.take(5)

([1.5779571530300995,
  1.7357597669255935,
  1.5345134073263487,
  1.3663369868130295,
  1.29015262921757],
 [0.14999, 0.225, 0.388, 0.394, 0.396])

In [30]:
# Zip `predictions` and `labels` into a list
predictionAndLabel = predictions.zip(labels).collect()

# Print out first 5 instances of `predictionAndLabel` 
predictionAndLabel[:5]

[(1.5779571530300995, 0.14999),
 (1.7357597669255935, 0.225),
 (1.5345134073263487, 0.388),
 (1.3663369868130295, 0.394),
 (1.29015262921757, 0.396)]

# Evaluating Model

In [31]:
# Coefficients for the model
linearModel.coefficients

DenseVector([0.0, 0.0, 0.0, 0.2744, 0.0, 0.0, 0.0])

In [32]:
# Intercept for the model
linearModel.intercept

1.0016345631509045

In [33]:
# Get the RMSE
linearModel.summary.rootMeanSquaredError


0.8770842163774836

In [34]:
# Get the R2
linearModel.summary.r2

0.4188005371043194

#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)